In [ ]:
from markdownify import markdownify as md
from markdownify import MarkdownConverter

import os
os.environ["EXECUTOR_URL"] = "***REMOVED***"
os.environ["EXECUTOR_AUTH"] = "***REMOVED***"

In [ ]:
import requests
from bs4 import BeautifulSoup
import html
import re

def get_codeforces_problem(contest_id, problem_index):
    api_url = f"https://codeforces.com/api/contest.standings?contestId={contest_id}&from=1&count=1&showUnofficial=true"
    response = requests.get(api_url)
    
    if response.status_code != 200:
        return f"Failed to fetch the problem. Status code: {response.status_code}"
    
    data = response.json()
    if data['status'] != 'OK':
        return f"API error: {data['comment']}"
    
    problems = data['result']['problems']
    problem = next((p for p in problems if p['index'] == problem_index), None)
    
    if not problem:
        return f"Problem {problem_index} not found in contest {contest_id}"
    
    problem_data = {
        'name': f"{problem['index']}. {problem['name']}",
        'rating': problem.get('rating', 'Not rated'),
        'tags': problem.get('tags', [])
    }
    
    # Fetch problem statement and samples
    problem_url = f"https://codeforces.com/contest/{contest_id}/problem/{problem_index}"
    response = requests.get(problem_url)
    
    if response.status_code != 200:
        return f"Failed to fetch problem details. Status code: {response.status_code}"
    
    soup = BeautifulSoup(response.text, 'html.parser')
    problem_statement = soup.find('div', class_='problem-statement')
    
    if problem_statement:
        # Extract the second direct child div in "problem-statement" and join paragraphs with double newlines
        problem_statement_divs = problem_statement.find_all('div', recursive=False)
        if len(problem_statement_divs) > 1:
            second_div = problem_statement_divs[1]
            problem_data["statement"] = MarkdownConverter().convert_soup(second_div)
            problem_data['statement'] = re.sub(r'\$\$\$(.*?)\$\$\$', r'\1', problem_data['statement'])  # Remove LaTeX dollar signs
        else:
            problem_data['statement'] = "Statement not found"
        # Parse time limit
        time_limit_text = problem_statement.find('div', class_='time-limit').text.strip()
        time_limit_match = re.search(r'(\d+) second', time_limit_text)
        if time_limit_match:
            problem_data['time_limit'] = int(time_limit_match.group(1))
        else:
            problem_data['time_limit'] = None
        
        input_spec = problem_statement.find('div', class_='input-specification')
        output_spec = problem_statement.find('div', class_='output-specification')
        notes = problem_statement.find('div', class_='note')

        problem_data['input_spec'] = MarkdownConverter().convert_soup(input_spec) if input_spec else "Not specified"
        problem_data['input_spec'] = re.sub(r'\$\$\$(.*?)\$\$\$', r'\1', problem_data['input_spec'])  # Remove LaTeX dollar signs
        problem_data['output_spec'] = MarkdownConverter().convert_soup(output_spec) if output_spec else "Not specified"
        problem_data['output_spec'] = re.sub(r'\$\$\$(.*?)\$\$\$', r'\1', problem_data['output_spec'])  # Remove LaTeX dollar signs
        problem_data['note'] = MarkdownConverter().convert_soup(notes) if notes else ""
        problem_data['note'] = re.sub(r'\$\$\$(.*?)\$\$\$', r'\1', problem_data['note'])  # Remove LaTeX dollar signs

        # Remove extra "Input" and "Output" headings
        problem_data['input_spec'] = re.sub(r'^Input', '', problem_data['input_spec']).strip()
        problem_data['output_spec'] = re.sub(r'^Output', '', problem_data['output_spec']).strip()
        sample_test = problem_statement.find('div', class_='sample-test')
        problem_data['samples'] = []
      
        input_divs = sample_test.find_all('div', class_='input')
        output_divs = sample_test.find_all('div', class_='output')
        
        for input_div, output_div in zip(input_divs, output_divs):
            # Process input data
            input_lines = input_div.find('pre').find_all('div', class_='test-example-line')
            input_data = '\n'.join(html.unescape(line.text.strip()) for line in input_lines)
            
            # Process output data
            output_data = output_div.find('pre').text.strip()
            output_data = html.unescape(output_data)
            
            problem_data['samples'].append({'input': input_data + "\n", 'output': output_data + "\n"})
       
        # Combine into problem_str
        problem_str = f"{problem_data['statement']}\n\nInput\n\n{problem_data['input_spec']}\n\nOutput\n\n{problem_data['output_spec']}\n\n"
        for i, sample in enumerate(problem_data['samples']):
            problem_str += f"Sample Input {i + 1}\n\n{sample['input']}\nSample Output {i + 1}\n\n{sample['output']}\n"
        problem_str += problem_data['note']
        problem_data['problem_str'] = problem_str.strip()
    
    return problem_data

In [ ]:
# Example usage
from base_classes import Problem, Test

contest_id = 1992
problem_index = "B"
problem_info = get_codeforces_problem(contest_id, problem_index)

public_tests = [Test(([sample["input"]], {}), sample["output"], None) for sample in problem_info["samples"]]
to_solve = Problem(problem_info["problem_str"], None, public_tests, None, None)

In [ ]:
import datetime
from basic_prompting import SimplePromptModel 
from simple_idea_model import SimpleIdeaModel
from simple_filter_models import SimpleFilteringModel
import os
from pathlib import Path


experiment_directory = f"codeforces/logs/{contest_id}_{problem_index}_{datetime.datetime.now().strftime('%m-%dT%H:%M:%S')}"
save_path = os.path.join(experiment_directory, "output.py")
model = SimpleIdeaModel(idea_model="gpt-4-turbo", code_model="gpt-4-turbo", experiment_directory=experiment_directory, cache_file="codeforces/cache.json", idea_temperature=0.5, code_temperature=0.5)
model = SimpleFilteringModel("filtering", experiment_directory, "codeforces/cache.json", model)
# Make the directory if it doesn't exist
Path(experiment_directory).mkdir(parents=True, exist_ok=True)
out_code = model.generate_solutions([to_solve])[0]
out_code += "\n#\n"

with open(save_path, "w") as f:
    f.write(out_code)
print(save_path)

print(f"{os.path.abspath(save_path)}")  # Add this line to display the full file path

from IPython.display import FileLink
# Display a link to download the file
print(FileLink(save_path))

In [ ]:
from exec_utils import run_tests
run_tests(out_code, public_tests, 30)

In [ ]:
!cf submit -f {save_path} {contest_id}{problem_index}